# Introduction

EV charging demand projection by county in New York State.

# Packages

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from EVIProLite_LoadPlotting import (temp_run, loadPlotting)

In [2]:
# Set up directories
cwd = os.getcwd()
input_dir = os.path.join(cwd, 'InputData')
output_dir = os.path.join(cwd, 'OutputData')
fig_dir = os.path.join(cwd, 'Figures')
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

print('Input directory: {}'.format(input_dir))
print('Output directory: {}'.format(output_dir))
print('Figure directory: {}'.format(fig_dir))

Input directory: d:\EERL\EV\EVI-Pro-Lite\InputData
Output directory: d:\EERL\EV\EVI-Pro-Lite\OutputData
Figure directory: d:\EERL\EV\EVI-Pro-Lite\Figures


In [3]:
# Set up API key
api_key_file = 'nrel_api_key.txt'

if os.path.isfile(api_key_file):
    # Read API key from file
    with open(api_key_file, 'r') as f:
        api_key = f.read()
        print('API key found in file.')
else:
    print('API key not found in file. Please enter your API key below.')
    api_key = input('Enter API key: ')

# Check if API key is valid
if len(api_key) != 40:
    Warning('API key is not valid. Use the demo key instead.')
    api_key = 'DEMO_KEY'

API key found in file.


# Read vehicle count data

In [4]:
vehicle_by_county_proj = pd.read_csv(os.path.join(input_dir, 'vehicle_by_county_proj.csv'),
                                     index_col=0)

# Use year 2035 as an example
year = 2035
vehicle_by_county_proj_year = vehicle_by_county_proj[str(year)]

# Rank counties in alphabetical order
vehicle_by_county_proj_year = vehicle_by_county_proj_year.sort_index()

vehicle_by_county_proj_year

County
Albany County         217592
Allegany County        33932
Bronx County          272253
Broome County         135672
Cattaraugus County     55522
                       ...  
Washington County      53558
Wayne County           80627
Westchester County    630513
Wyoming County         33114
Yates County           18817
Name: 2035, Length: 63, dtype: int64

# Read temperature data

In [5]:
# Hourly air temperature data in 2018 from NREL ResStock
temp_df = pd.read_csv(os.path.join(input_dir, 'resstock_amy2018_temp.csv'), 
                      index_col=0, parse_dates=True)

# Parse county names and rename columns
county_names = [county.split(',')[1] for county in temp_df.columns]
county_names = [county.strip() for county in county_names]
temp_df.columns = county_names
temp_df

,Albany County,Allegany County,Bronx County,Broome County,Cattaraugus County,Cayuga County,Chautauqua County,Chemung County,Chenango County,Clinton County,...,Sullivan County,Tioga County,Tompkins County,Ulster County,Warren County,Washington County,Wayne County,Westchester County,Wyoming County,Yates County
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,-19.400000,-17.707692,-12.015385,-19.400000,-21.825000,-17.100000,-12.006667,-19.4,-19.400000,-21.007692,...,-19.400,-19.400000,-19.400000,-20.6,-19.800000,-19.800000,-16.10,-14.40,-16.100000,-16.10
2018-01-01 01:00:00,-20.000000,-17.092308,-12.092308,-19.550000,-21.028571,-17.000000,-12.169231,-20.0,-19.550000,-22.446154,...,-20.000,-19.550000,-19.550000,-21.7,-20.000000,-20.000000,-15.80,-14.40,-16.100000,-15.80
2018-01-01 02:00:00,-22.200000,-17.285714,-12.600000,-20.000000,-20.232143,-17.625000,-11.976923,-18.9,-20.000000,-22.830769,...,-21.100,-20.000000,-20.000000,-21.7,-20.460000,-20.460000,-15.60,-14.40,-16.100000,-15.60
2018-01-01 03:00:00,-22.200000,-17.864286,-12.984615,-19.700000,-17.864286,-19.150000,-11.976923,-19.7,-19.700000,-22.361538,...,-21.100,-19.700000,-19.700000,-22.2,-21.733333,-21.733333,-15.60,-15.00,-16.400000,-15.60
2018-01-01 04:00:00,-21.400000,-18.100000,-12.984615,-20.000000,-18.100000,-19.760000,-12.253846,-19.7,-20.000000,-22.746154,...,-20.000,-20.000000,-20.000000,-21.7,-23.230000,-23.230000,-16.10,-15.00,-16.700000,-16.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,3.900000,3.023077,6.976923,2.200000,3.900000,3.300000,6.753846,5.6,2.200000,3.023077,...,1.700,2.200000,2.200000,4.4,0.976923,0.976923,4.60,4.40,5.766667,4.60
2018-12-31 20:00:00,3.900000,6.207692,7.026667,2.800000,5.000000,3.300000,8.700000,6.7,2.800000,2.784615,...,1.900,2.800000,2.800000,4.2,1.009091,1.009091,6.10,5.00,5.600000,6.10
2018-12-31 21:00:00,3.900000,7.015385,7.471429,3.300000,4.400000,3.300000,11.292308,7.2,3.300000,2.015385,...,1.925,3.300000,3.300000,5.0,0.927273,0.927273,6.10,5.80,9.150000,6.10


# Set up simulation parameters

In [6]:
def county_run(county, temp_csv, scenario_csv, api_key):
    """
    Run the EVI-Pro Lite model for a county.

    Parameters
    ----------
    county : str
        County name.
    temp_csv : pandas.DataFrame
        Hourly temperature data.
    scenario_csv : pandas.DataFrame
        Scenario data.
    api_key : str
        NREL API key.

    Returns
    -------
    None.
    """
    
    temp_csv['date'] = temp_csv['date'].dt.date
    # Saturday and Sunday are 5 and 6, Monday is 0. <5 is weekday
    temp_csv['weekday'] = temp_csv['date'].apply(lambda x: x.weekday())#<5)
    temp_csv['temp_c'] = temp_csv['temperature']
    temp_csv.drop('temperature',axis = 1,inplace=True)

    # Run the model
    final_result = temp_run(scenario_csv,temp_csv,api_key)
    
    # Plotting and Save CSVs with data
    for scenario,row in scenario_csv.iterrows():
        # Plot charging demand for the first week
        fig_name = os.path.join(fig_dir, f'{county}_scen{str(scenario)}_temp_gridLoad.png'.replace(' ','_'))
        loadPlotting(final_result, scenario, fig_name)

        # Save charging demand to CSV
        filename = os.path.join(output_dir, f'{county}_scen{str(scenario)}_temp_gridLoad.csv'.replace(' ','_'))
        final_result[scenario].to_csv(filename)
    
    return None

Input values for the EVI-Pro Lite model
```python
param_dict = {
    "fleet_size" : [50000,10000,1000],
    "mean_dvmt" : [45,35,25],
    "temp_c" : [40,30,20,10,0,-10,-20],
    "pev_type" : ['PHEV20','PHEV50','BEV100','BEV250'],
    "pev_dist" : ['BEV','PHEV','EQUAL'],
    "class_dist" : ['Sedan','SUV','Equal'],
    "home_access_dist" : ['HA100','HA75','HA50',],
    "home_power_dist" : ['MostL1','MostL2','Equal'],
    "work_power_dist" : ['MostL1','MostL2','Equal'],
    "pref_dist" : ['Home60','Home80','Home100'],  
    "res_charging" : ['min_delay','max_delay','midnight_charge'],
    "work_charging" : ['min_delay','max_delay']
}
```

In [10]:
# Calculate daily average temperature
temp_df_daily = temp_df.resample('D').mean()

# Only run for the first 10 days
temp_df_daily = temp_df_daily.iloc[:10]

# Loop through all counties
for county in county_names[:3]:
    print("Running for county: ", county)

    # Get daily average temperature for the county
    temp_csv = pd.DataFrame(temp_df_daily[county]).reset_index()
    temp_csv.columns = ['date', 'temperature']

    # Get scenario data for the county
    fleet_size = vehicle_by_county_proj_year[county]
    temp_c = temp_csv['temperature'].mean()

    # Example scenario: two PHEV types
    # User can change the scenario data here
    scenario_dict = {
        'fleet_size': [fleet_size]*2,
        'mean_dvmt': [35]*2,
        'temp_c': [temp_c]*2,
        'pev_type': ['PHEV20', 'PHEV50'],
        'pev_dist': ['EQUAL']*2,
        'class_dist': ['Equal']*2,
        'home_access_dist': ['HA75']*2,
        'home_power_dist': ['MostL1']*2,
        'work_power_dist': ['MostL2']*2,
        'pref_dist': ['Home60']*2,
        'res_charging': ['min_delay']*2,
        'work_charging': ['min_delay']*2,
    }
    scenario_csv = pd.DataFrame(scenario_dict)
    # Save scenario data to CSV
    scenario_csv.to_csv(os.path.join(output_dir, f'{county}_scenarios.csv'.replace(' ','_')))

    # Run the model
    county_run(county, temp_csv, scenario_csv, api_key)

    print('Done!')


Running for county:  Albany County
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
Done!
Running for county:  Allegany County
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
Done!
Running for county:  Bronx County
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
2018-01-01
2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06
2018-01-07
2018-01-08
2018-01-09
2018-01-10
Done!
